In [1]:
#import libraries
import boto3
import csv
import json
import time

In [7]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
# folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
# file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
prefix = "english_documents/raw_pdf/"
i = 0
for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
    srcKey = object.key
    if ".txt" in srcKey:
        print(i)
        newKey = srcKey.replace(".txt", ".pdf")
        copySource = s3BucketName + '/' + srcKey         
        s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
        s3.Object(s3BucketName, srcKey).delete()
    i += 1

3
5
6
8
9
11
12
13
14
15
17
20
22
23
25
26
27
28
29
30
31
38
39
41
42
43
44
45
46
47
48
49
50
54
57
58
59
61
62
63
69
73
75
76
77
80
81
82
83
84
85
86
87
88
89
95
97
98
101
103
104
106
107
108
110
111
112
113
114
115
116
118
119
120
123
124
126
128
131
135
136
138
139
140
142
143
144
146
150
151
155
163
165
166
168
170
171
174
175
180
181
184
185
186
188
189
191
192
194
195
197
198
199
201
202
203
204
205
206
209
210
213
217
221
222
224
228
229
231
236
237
239
240
243
244
245
246
247
250
251
252
256
257
260
266
267
268
269
270
271
273
274
276
279
280
281
283
284
285
286
291
292
294
300
301
302
304
306
307
309
312
313
315
316
319
320
322
323
324
325
326
327
328
331
332
333
335
336
337
339
340
341
342
345
346
347
352
354
356
358
359
361
363
368
369
370
373
374
376
377
378
379
380
382
386
387
390
393
399
400
404
405
406
407
408
409
410
411
412
413
418
419
420
421
422
425
426
427
429
431
432
434
435
438
439
440
442
443
444
449
453
455
456
457
458
460
464
469
470
472
473
474
475
476
477
480

In [4]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [5]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

In [9]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "spanish_documents/raw_pdf_updated/"
i = 0

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key:# and i > 48: lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        key = obj.key.replace("raw_pdf_updated", "text_files/HSSC/new").replace(".pdf", ".txt")
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        if policy != "":
            s3.Object(s3BucketName, key).put(Body = policy)
#         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

1 ** spanish_documents/raw_pdf_updated/01dcc9c9e259dce1ba890071a50b05082bf93391.pdf
2 ** spanish_documents/raw_pdf_updated/029d4119e3f7ead62dd282e72116ec353ff32847.pdf
3 ** spanish_documents/raw_pdf_updated/04731acfcc33150107e43e581eba915c1082bff5.pdf
4 ** spanish_documents/raw_pdf_updated/05149e3e1e7e88ee226a245ad292a5c72da739e7.pdf
5 ** spanish_documents/raw_pdf_updated/069d9e560c0560253f9430598a859f7c7fc3e69c.pdf
6 ** spanish_documents/raw_pdf_updated/074b95f242852b684228b32183f8d2f799e1d4c6.pdf
7 ** spanish_documents/raw_pdf_updated/07d1de22699ca40b61fccff898de1a572f9c2245.pdf
8 ** spanish_documents/raw_pdf_updated/08177646bb8afadcaebf4423e42009409879b553.pdf
9 ** spanish_documents/raw_pdf_updated/08f5a1a53398dc6d2d3add60be3cbc1ca4ab0d9e.pdf
10 ** spanish_documents/raw_pdf_updated/0cffb35cd25d9440b1ac33c5bc98a31af423f5b3.pdf
11 ** spanish_documents/raw_pdf_updated/0d090c50af01109dc3849c1a43edd95bbdcc67df.pdf
12 ** spanish_documents/raw_pdf_updated/0d471c6eb518dfb33416f37b35a3913325

98 ** spanish_documents/raw_pdf_updated/6f2b5a53e5a49fc3cc920e4df139a919e87e292c.pdf
99 ** spanish_documents/raw_pdf_updated/708b180a85891f1f010882f4d4b051a03bf3b932.pdf
100 ** spanish_documents/raw_pdf_updated/731dbf06b067087a6c2b51ed5b4c839da9d39fe4.pdf
101 ** spanish_documents/raw_pdf_updated/732ad5f5bb5779bc4f04d237074eae79b7e85126.pdf
102 ** spanish_documents/raw_pdf_updated/7401043dedcb3383481401e0ab64867f5dc226e8.pdf
103 ** spanish_documents/raw_pdf_updated/753300f5baf327a582ddce02175c9ba42da793fb.pdf
104 ** spanish_documents/raw_pdf_updated/7620569d01ed0540246f93c7041e11ea2f2d91b1.pdf
105 ** spanish_documents/raw_pdf_updated/76b6e7f634dd305377eec41834eb34ed948d2d16.pdf
106 ** spanish_documents/raw_pdf_updated/78a302a2752a023ae96b2749981d0c7e4c316163.pdf
107 ** spanish_documents/raw_pdf_updated/7951d1c0525c05bc4c247b290dbefcf64f93b947.pdf
108 ** spanish_documents/raw_pdf_updated/7c850afe065281bb05d3e2c5b3442aba58bd97ac.pdf
109 ** spanish_documents/raw_pdf_updated/7d1f7bb44252034

194 ** spanish_documents/raw_pdf_updated/cebf6a962ecefbd48ca22d1be7c4ebc914faf7f8.pdf
195 ** spanish_documents/raw_pdf_updated/d059638adec701a2a43535807d5283bd0101aed2.pdf
196 ** spanish_documents/raw_pdf_updated/d1b8337f623164ff191d83c9537fc2ab826770a4.pdf
197 ** spanish_documents/raw_pdf_updated/d1d92204153110df678b006b701b07306293b6da.pdf
198 ** spanish_documents/raw_pdf_updated/d2bbc3490e1256be6b04af2ea004818c9610f0b6.pdf
199 ** spanish_documents/raw_pdf_updated/d44820bdbe1ca89f0a19b25fbe3420ab9014b60d.pdf
200 ** spanish_documents/raw_pdf_updated/d56d9112281281d422c8d1966776bff734ce89b0.pdf
201 ** spanish_documents/raw_pdf_updated/d7635349720ca4fbf3f96da48c149466ffaa95bf.pdf
202 ** spanish_documents/raw_pdf_updated/d79b35db0455559d21d29e4d42b61386e45f0309.pdf
203 ** spanish_documents/raw_pdf_updated/daafe7495eebf7ad6635bc90c04cce6680eb8acb.pdf
204 ** spanish_documents/raw_pdf_updated/de02487a8e568edef5a52e9898932ff5209b6500.pdf
205 ** spanish_documents/raw_pdf_updated/df1255e308e5e

In [ ]:
policy